In [9]:
import pandas as pd
import plotly.express as px
from dash import dcc
import numpy as np

In [3]:
spacex_df = pd.read_csv("data/spacex_launch_dash.csv")
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [4]:
sites = spacex_df["Launch Site"].unique()

options = [{'label': 'All Sites', 'value': 'ALL'}] 
for site in spacex_df["Launch Site"].unique(): 
    options.append({ 'label':site, 'value':site })

sites[0]

'CCAFS LC-40'

In [10]:
entered_site = 'CCAFS LC-40'

filtered_df = spacex_df[spacex_df["Launch Site"]==entered_site]
grouped_df = filtered_df.groupby('class').size().reset_index(name='counts')

fig = px.pie(grouped_df, values='counts', 
    names= 'class' , 
    title='Total Success Launches for site ' + entered_site )
fig.show()

dcc.RangeSlider(id='payload-slider',
                min=0, max=10000, step=1000,
                marks={0: '0', 100: '100'},
                value=[np.min(spacex_df[['Payload Mass (kg)']]), 
                       np.max(spacex_df[['Payload Mass (kg)']])
                       ]
                )

                                

RangeSlider(min=0, max=10000, step=1000, marks={0: '0', 100: '100'}, value=[0.0, 9600.0], id='payload-slider')

In [19]:
(spacex_df[['Payload Mass (kg)']]).max()

Payload Mass (kg)    9600.0
dtype: float64